<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/devops_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j openai langchain tiktoken

In [2]:
from langchain.graphs import Neo4jGraph

url = "neo4j+s://databases.neo4j.io"
username ="neo4j"
password = ""
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [3]:
import requests
import_url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/33221f402be58fdd61bebc451f37113e173d7741/microservices.json"
import_query = requests.get(import_url).json()['query']
graph.query(
    import_query
)

[]

In [24]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings

os.environ['OPENAI_API_KEY'] = "sk-"

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

In [25]:
response = vector_index.similarity_search(
    "How will RecommendationService be updated?"
)
print(response[0].page_content)


name: RecommendationFeature
description: Add a new feature to RecommendationService to provide more personalized and accurate product recommendations to the users, leveraging user behavior and preference data.
status: In Progress


In [27]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(), chain_type="stuff", retriever=vector_index.as_retriever())

In [28]:
vector_qa.run(
    "How will recommendation service be updated?"
)

'The RecommendationService is currently in progress of being updated with a new feature. The new feature aims to provide more personalized and accurate product recommendations to the users by leveraging user behavior and preference data. However, the specific details of how the service will be updated are not provided in the context.'

In [29]:
vector_qa.run(
    "How many open tickets there are?"
)

'There are 4 open tickets.'

In [9]:
graph.query(
    "MATCH (t:Task {status:'Open'}) RETURN count(*)"
)

[{'count(*)': 5}]

In [10]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4'),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [11]:
chain.run(
    "How many Open tickets there are?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task {status: 'Open'}) RETURN COUNT(t)
Full Context:
[{'COUNT(t)': 5}]

> Finished chain.


'There are 5 Open tickets.'

In [12]:
chain.run(
    "Which team has the most Open tasks?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task)-[:ASSIGNED_TO]->(team:Team)
WHERE t.status = 'Open'
RETURN team.name, COUNT(t) AS OpenTasks
ORDER BY OpenTasks DESC
LIMIT 1
Full Context:
[{'team.name': 'TeamA', 'OpenTasks': 3}]

> Finished chain.


'TeamA has the most open tasks with a total of 3.'

In [13]:
chain.run(
    "Which services depend on Database directly?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice)-[:DEPENDS_ON]->(n:Microservice {name: 'Database'}) RETURN m.name
Full Context:
[{'m.name': 'PaymentService'}, {'m.name': 'OrderService'}, {'m.name': 'InventoryService'}, {'m.name': 'CatalogService'}, {'m.name': 'UserService'}, {'m.name': 'AuthService'}]

> Finished chain.


'The services that depend on the Database directly are the PaymentService, OrderService, InventoryService, CatalogService, UserService, and AuthService.'

In [14]:
chain.run(
    "Which services depend on Database indirectly?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice)-[:DEPENDS_ON*2..]->(d:Microservice {name: 'Database'}) RETURN m.name
Full Context:
[{'m.name': 'OrderService'}, {'m.name': 'OrderService'}, {'m.name': 'ShippingService'}, {'m.name': 'OrderService'}, {'m.name': 'PaymentService'}, {'m.name': 'OrderService'}, {'m.name': 'UserService'}]

> Finished chain.


'The services that depend on the Database indirectly are the OrderService, ShippingService, PaymentService, and UserService.'